In [ ]:
# First install required libraries if not already done
!pip install transformers datasets scikit-learn torch pandas matplotlib seaborn jupyter kagglehub

In [3]:
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    acc = accuracy_score(labels, preds)
    return {
        "accuracy": acc,
        "precision": precision,
        "recall": recall,
        "f1": f1
    }


In [ ]:


import kagglehub
import pandas as pd
import os
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from datasets import Dataset

# Download dataset
path = kagglehub.dataset_download("ankurzing/sentiment-analysis-for-financial-news")
print("Path to dataset files:", path)

# Define file path
file_path = os.path.join(path, "all-data.csv")

# Read CSV with no header and proper encoding
df = pd.read_csv(file_path, engine='python', sep=',', header=None, encoding='latin-1')

# Check the first few rows to understand column order
print("First 5 rows:")
print(df.head())

# Assign column names manually based on inspection
# Based on public dataset info, it's usually [sentiment, text]
df.columns = ['label', 'text']

# Now filter valid labels
valid_labels = {'positive', 'neutral', 'negative'}
df['label'] = df['label'].str.strip()
df = df[df['label'].isin(valid_labels)]

# Reset index
df.reset_index(drop=True, inplace=True)

# Split data
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# Optional: print shapes
print("\nTrain shape:", train_df.shape)
print("Test shape:", test_df.shape)

model_name = "yiyanghkust/finbert-tone"  # FinBERT
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Convert pandas DataFrames to Dataset objects
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

# Define your label mappings correctly (only the valid ones)
labels = ['positive', 'neutral', 'negative']

# Create mappings
label2id = {label: i for i, label in enumerate(labels)}
id2label = {i: label for label, i in label2id.items()}

print("label2id:", label2id)
print("id2label:", id2label)

def tokenize_function(examples):
    # Map string labels to integers within the tokenization function
    # Ensure examples['label'] is a string before mapping
    label_str = examples["label"]
    examples["labels"] = label2id[label_str]
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=512)

tokenized_train = train_dataset.map(tokenize_function, batched=False)
tokenized_test = test_dataset.map(tokenize_function, batched=False)

# Remove the original 'label' column as we now have 'labels'
tokenized_train = tokenized_train.remove_columns(["label"])
tokenized_test = tokenized_test.remove_columns(["label"])

model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=len(labels), # Use len(labels) to be dynamic
    id2label=id2label,
    label2id=label2id
)

training_args = TrainingArguments(
    output_dir="finbert-finetuned",
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=18,
    per_device_eval_batch_size=18,
    num_train_epochs=5,
    weight_decay=0.01,
    save_strategy="epoch",
    load_best_model_at_end=True,
    logging_dir="logs",
    logging_strategy="epoch",
    save_total_limit=2,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    report_to="tensorboard",
    optim="adamw_torch",
    lr_scheduler_type="linear",  # 👈 corrected scheduler
    warmup_steps=500,
    seed=42,
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,    # ← here
)


trainer.train()
model.save_pretrained("finbert-finetuned-final")
tokenizer.save_pretrained("finbert-finetuned-final")





In [ ]:
import kagglehub
import pandas as pd
import os
from datasets import load_dataset, Dataset
from sklearn.model_selection import train_test_split

# ---------------------------------------------------------
# 1. Load and Prep Kaggle Dataset (Financial PhraseBank)
# ---------------------------------------------------------
path = kagglehub.dataset_download("ankurzing/sentiment-analysis-for-financial-news")
kaggle_file_path = os.path.join(path, "all-data.csv")

# Read Kaggle CSV
df_kaggle = pd.read_csv(kaggle_file_path, engine='python', sep=',', header=None, encoding='latin-1')
df_kaggle.columns = ['label', 'text'] # Rename to standard columns

# Filter only valid labels
df_kaggle['label'] = df_kaggle['label'].str.strip()
df_kaggle = df_kaggle[df_kaggle['label'].isin(['positive', 'neutral', 'negative'])]

print(f"Kaggle rows: {len(df_kaggle)}")

# ---------------------------------------------------------
# 2. Load and Prep FiQA Dataset
# ---------------------------------------------------------
fiqa_data = load_dataset("TheFinAI/fiqa-sentiment-classification")
df_fiqa = fiqa_data["train"].to_pandas()  # Using 'train' split for more data

# Define conversion logic (Modified to return STRINGS to match Kaggle)
def score_to_label_str(score):
    # Using 0.0 as cutoff as requested, but returning strings
    if score > 0.0:
        return "positive"
    elif score < -0.0:
        return "negative"
    else:
        return "neutral"

# Apply conversion
df_fiqa['label'] = df_fiqa['score'].apply(score_to_label_str)
df_fiqa = df_fiqa.rename(columns={'sentence': 'text'})

# Keep only the columns we need
df_fiqa = df_fiqa[['label', 'text']]

print(f"FiQA rows: {len(df_fiqa)}")

# ---------------------------------------------------------
# 3. MERGE AND SHUFFLE (The Critical Step)
# ---------------------------------------------------------

# Concatenate both dataframes
combined_df = pd.concat([df_kaggle, df_fiqa], ignore_index=True)

# Shuffle the rows randomly (frac=1 means sample 100% of rows)
# random_state=42 ensures reproducible shuffling
combined_df = combined_df.sample(frac=1, random_state=42).reset_index(drop=True)

print(f"Total Combined rows: {len(combined_df)}")
print("First 5 rows of mixed data:")
print(combined_df.head())

# ---------------------------------------------------------
# 4. Resume Your Standard Pipeline
# ---------------------------------------------------------

# Split data (now containing a mix of both sources)
train_df, test_df = train_test_split(combined_df, test_size=0.2, random_state=42)

# Convert to Hugging Face Datasets
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

print("\nReady for tokenization and training!")

# 1. Define Labels
labels = ['positive', 'neutral', 'negative']
model_name = "yiyanghkust/finbert-tone"  # FinBERT
#tokenizer = AutoTokenizer.from_pretrained(model_name)

# 2. Create Mappings
label2id = {label: i for i, label in enumerate(labels)}
id2label = {i: label for label, i in label2id.items()}

print("label2id:", label2id)
print("id2label:", id2label)

# 3. Tokenize with the new mappings
def tokenize_function(examples):
    label_str = examples["label"]
    examples["labels"] = label2id[label_str]
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=512)

tokenized_train = train_dataset.map(tokenize_function, batched=False)
tokenized_test = test_dataset.map(tokenize_function, batched=False)

# 4. Clean Columns
tokenized_train = tokenized_train.remove_columns(["label"])
tokenized_test = tokenized_test.remove_columns(["label"])

# 5. Load Model
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=len(labels),
    id2label=id2label,
    label2id=label2id
)

# 6. Training Arguments
training_args = TrainingArguments(
    output_dir="finbert-combined-finetuned", # Changed name to reflect combined data
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=18,
    per_device_eval_batch_size=18,
    num_train_epochs=3, # Reduced to 3 as 5 might overfit with more data
    weight_decay=0.01,
    save_strategy="epoch",
    load_best_model_at_end=True,
    logging_dir="logs",
    logging_strategy="epoch",
    save_total_limit=2,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    report_to="tensorboard",
    optim="adamw_torch",
    lr_scheduler_type="linear",
    warmup_steps=500,
    seed=42,
)

# 7. Initialize Trainer
# NOTE: The 'compute_metrics' function here typically uses np.argmax(logits).
# This works because the highest logit ALWAYS corresponds to the highest Softmax probability.
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics, 
)

# 8. Train and Save
trainer.train()
model.save_pretrained("finbert-combined-final")
tokenizer.save_pretrained("finbert-combined-final")

In [ ]:
# Zip & download your saved model folder in Colab

import shutil
from google.colab import files

# Define the correct directory name
directory_to_zip = "finbert-base-model"

# 1. Make a .zip archive of the saved model directory
shutil.make_archive(
    base_name=directory_to_zip,  # The name for the output .zip file
    format="zip",
    root_dir=directory_to_zip    # The folder you want to zip
)

# 2. Download it to your local machine
print(f"✅ Archiving complete. Downloading {directory_to_zip}.zip...")
files.download(f"{directory_to_zip}.zip")